In [41]:
#!pip install polars

In [2]:
import pandas as pd
import numpy as np
import polars as pl
import ast

In [2]:
### Read csv file
df = pd.read_csv('./csv/nrw_hai_extracted.csv')

In [3]:
df.nunique()

AGG_DAY_PERIOD              366
BOUNDS                  3036097
XLAT                       3072
XLON                       2896
ACTIVITY_INDEX_TOTAL     390247
dtype: int64

In [4]:
len(df)

52343230

In [8]:
#x_df=df.groupby(['AGG_DAY_PERIOD', 'BOUNDS'], as_index=False).mean()
del x_df

In [9]:
# Group by AGG_DAY_PERIOD and BOUNDS and compute the mean ACTIVITY_INDEX_TOTAL
grouped_df = df.groupby(['AGG_DAY_PERIOD', 'BOUNDS'], as_index=False).agg({'ACTIVITY_INDEX_TOTAL': 'mean'})#.rename(columns={'ACTIVITY_INDEX_TOTAL': 'MEAN_ACTIVITY_INDEX_TOTAL'})

In [35]:
grouped_df.head(100000000000)[50000000:50000009]

,AGG_DAY_PERIOD,BOUNDS,ACTIVITY_INDEX_TOTAL
50000000,2020-12-10,"[\n 9.518280029296875e+00,\n 5.2347085391106...",0.064982
50000001,2020-12-10,"[\n 9.518280029296875e+00,\n 5.2407445753192...",0.247550
50000002,2020-12-10,"[\n 9.518280029296875e+00,\n 5.2414147367821...",0.019615
50000003,2020-12-10,"[\n 9.518280029296875e+00,\n 5.2426710150871...",0.104839
50000004,2020-12-10,"[\n 9.518280029296875e+00,\n 5.2434246102623...",0.018592
50000005,2020-12-10,"[\n 9.518280029296875e+00,\n 5.2457683046545...",0.069178
50000006,2020-12-10,"[\n 9.518280029296875e+00,\n 5.2458519849642...",0.069772
50000007,2020-12-10,"[\n 9.518280029296875e+00,\n 5.2461866903001...",0.179577
50000008,2020-12-10,"[\n 9.519653320312500e+00,\n 5.1267929925864...",0.159727


In [11]:
grouped_df.nunique()

AGG_DAY_PERIOD              366
BOUNDS                  3036097
ACTIVITY_INDEX_TOTAL     514692
dtype: int64

In [17]:
len(grouped_df['ACTIVITY_INDEX_TOTAL'])
#grouped_df.to_csv('./csv/grouped_csv.csv')

52143374

In [3]:
df=pl.read_csv('./csv/grouped_csv.csv')

In [5]:
# Function to calculate the centroid of the bounds
def calculate_centroid(bounds):
    lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat = bounds
    centroid_lon = (lower_left_lon + upper_right_lon) / 2
    centroid_lat = (lower_left_lat + upper_right_lat) / 2
    return centroid_lat, centroid_lon

In [ ]:
# Apply the function to calculate centroids
df = df.with_columns(
    pl.col("BOUNDS").apply(lambda x: calculate_centroid(ast.literal_eval(x)), return_dtype=pl.List(pl.Float64)).alias("CENTROID")
)

C:\Users\janak\AppData\Local\Temp\ipykernel_19232\1482069414.py:3: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("BOUNDS").apply(lambda x: calculate_centroid(ast.literal_eval(x)), return_dtype=pl.List(pl.Float64)).alias("CENTROID")


In [ ]:
# Split the centroid into two separate columns
df = df.with_columns([
    pl.col("CENTROID").arr.get(0).alias("CENTROID_LAT"),
    pl.col("CENTROID").arr.get(1).alias("CENTROID_LON")
]).drop("CENTROID")


In [ ]:
# Save the resulting DataFrame to a new CSV file
df.write_csv('centroid_bounds.csv')

print("The CSV file with centroids has been created successfully.")

In [ ]:
import polars as pl
import ast

# Load the CSV file
df = pl.read_csv('your_file.csv')

# Function to calculate the centroid of the bounds
def calculate_centroid(bounds):
    lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat = bounds
    centroid_lon = (lower_left_lon + upper_right_lon) / 2
    centroid_lat = (lower_left_lat + upper_right_lat) / 2
    return centroid_lat, centroid_lon

# Apply the function to calculate centroids
df = df.with_column(
    pl.col("BOUNDS").apply(lambda x: calculate_centroid(ast.literal_eval(x)), return_dtype=pl.List(pl.Float64)).alias("CENTROID")
)

# Split the centroid into two separate columns
df = df.with_columns([
    pl.col("CENTROID").arr.get(0).alias("CENTROID_LAT"),
    pl.col("CENTROID").arr.get(1).alias("CENTROID_LON")
]).drop("CENTROID")

# Save the resulting DataFrame to a new CSV file
df.write_csv('centroid_bounds.csv')

print("The CSV file with centroids has been created successfully.")


In [37]:
# Parse the BOUNDS field and calculate the centroid for each bound
grouped_df['BOUNDS'] = grouped_df['BOUNDS'].apply(eval)  # Convert the string representation of the list to an actual list

MemoryError: 

In [ ]:
grouped_df[['CENTROID_LAT', 'CENTROID_LON']] = grouped_df['BOUNDS'].apply(lambda b: pd.Series(calculate_centroid(b)))

In [ ]:
# Function to calculate the centroid of the bounds
def calculate_centroid(bounds):
    lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat = bounds
    centroid_lon = (lower_left_lon + upper_right_lon) / 2
    centroid_lat = (lower_left_lat + upper_right_lat) / 2
    return centroid_lat, centroid_lon

# Parse the BOUNDS field and calculate the centroid for each bound
extracted_df['BOUNDS'] = extracted_df['BOUNDS'].apply(eval)  # Convert the string representation of the list to an actual list
extracted_df[['CENTROID_LAT', 'CENTROID_LON']] = extracted_df['BOUNDS'].apply(lambda b: pd.Series(calculate_centroid(b)))

# Group by AGG_DAY_PERIOD and BOUNDS and compute the mean ACTIVITY_INDEX_TOTAL
grouped_df = extracted_df.groupby(['AGG_DAY_PERIOD', 'BOUNDS', 'CENTROID_LAT', 'CENTROID_LON'], as_index=False).agg({
    'ACTIVITY_INDEX_TOTAL': 'mean'
}).rename(columns={'ACTIVITY_INDEX_TOTAL': 'MEAN_ACTIVITY_INDEX_TOTAL'})

# Save the resulting DataFrame to a new CSV file
grouped_df.to_csv('mean_activity_index_per_day_bounds.csv', index=False)

print("The CSV file has been created successfully.")
